In [1]:
import io
import os
import re
import sys
import ast
import pytz
import json
import time
import shutil
import pprint
import zipfile
import tarfile
import logging
import warnings
import datetime
import requests
import xml.etree.ElementTree

from IPython.display import HTML, Javascript, display
from ipywidgets.widgets import *
import pyunicore.restutils as unicore

In [2]:
path = "."
while not os.path.isdir(os.path.join(path, "CLS-INTERACTIVE-MODELS")):
    path = os.path.join(path, "..")
path = os.path.join(path, "CLS-INTERACTIVE-MODELS", "models.json")

with open(path) as json_file:
    data = json.load(json_file)
    f = data['singlecellmodeling']
    model_url = f[len(f) - 1]['name']

In [3]:
if os.path.isdir('GUIBPOPTONE'):
    shutil.rmtree('GUIBPOPTONE')

os.mkdir("GUIBPOPTONE")
os.chdir("GUIBPOPTONE")

In [4]:
h0=HTML('''<style>
    .widget-label { min-width: 17ex !important;}
</style>''')
h0.display='none'

javascript2 = """
<script type="text/Javascript">
    document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
    document.styleSheets[0].insertRule(".form_login > label { width: 30% }", 1);
    document.styleSheets[0].insertRule(".form_login > label + input { width: 70% }", 1);
</script>
"""
h1=HTML(javascript2)
h1.display='none'

In [5]:
warnings.filterwarnings("ignore")

In [6]:
listfilenames=[]
for filename in os.listdir(os.getcwd()):
    listfilenames.append(filename)
if 'Work-in-progress3.png' not in listfilenames:
    shutil.copy2(os.path.join('..','Work-in-progress3.png'), 'Work-in-progress3.png')
if 'unicore_api_Python3.ipynb' not in listfilenames:
    shutil.copy2(os.path.join('..','unicore_api_Python3.ipynb'), 'unicore_api_Python3.ipynb')
foldernameOPTstring = model_url

In [7]:
%%capture
URL=('https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/hippocampus_optimization/bsp_data_repository-4.0.5_Python3.tar.gz')
!wget "{URL}"
!tar -zxvf bsp_data_repository-4.0.5_Python3.tar.gz

In [8]:
diff = lambda l1,l2: [x for x in l1 if x not in l2]
os.chdir('bsp_data_repository-4.0.5_Python3')
os.chdir('optimizations')
FD=diff(os.listdir('.'),['README'])
for i in range(len(FD)):
    shutil.copytree(FD[i],os.path.join('..','..',FD[i]))
os.chdir('../..')

In [9]:
%run "unicore_api_Python3.ipynb"

In [10]:
for files in os.listdir(foldernameOPTstring):
    if files.endswith('.zip'):
        os.chdir(foldernameOPTstring)
        zip_ref = zipfile.ZipFile(files, 'r')
        zip_ref.extractall()
        zip_ref.close()          
os.chdir('..')

H1 = Textarea(layout=Layout(width='80%', height='200px'))
H2 = Textarea(layout=Layout(width='80%', height='200px'))
os.chdir(foldernameOPTstring)
os.chdir(foldernameOPTstring)
os.chdir('config')
with open('features.json') as data_file:    
    data = json.load(data_file)
H1.value=pprint.pformat(data[list(data.keys())[0]],width=80,depth=5)
H1.disabled=True
with open('parameters.json') as data_file:    
    data = json.load(data_file)
H2.value=pprint.pformat(data,width=120,depth=5)
os.chdir('../../../..')
BT1 = Button()
BT1.description = 'Save new parameters'
BT1.background_color='gainsboro'
BT1.border_color='black'

def saveparamfile(_):
    os.chdir('GUIBPOPTONE')
    os.chdir(foldernameOPTstring)
    os.chdir(foldernameOPTstring)
    os.chdir('config')
    with open('parameters.json','w') as f:
        json.dump(ast.literal_eval(H2.value),f)
    f.close()
    os.chdir('../../../..')
    H2.disabled=True
    BT1.disabled=True
BT1.on_click(saveparamfile)
page1 = Box(children=[H1])
page2 = Box(children=[VBox([H2,BT1],layout=Layout(width='100%'))])
tabs = Tab(children=[page1,page2])
tabs.set_title(0, 'Features')
tabs.set_title(1, 'Parameters')
display(tabs)


NGEN=Text(**dict(
        margin='0px 0px 0px 16px'
))
NGEN.description='max_ngen'
NGEN.value='2'
NGEN.width=600
display(NGEN)
OS=Text(**dict(
        margin='0px 0px 0px 16px'
))
OS.description='offspring_size'
OS.value='10'
OS.width=200
display(OS)

def createzip(_):
    os.chdir('GUIBPOPTONE')
    os.chdir(foldernameOPTstring)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith('init') or file.endswith('.zip'):
            os.remove(file)
    with open('init.py','w') as f:
        f.write('import os')
        f.write('\n')
        f.write('os.system(\'python opt_neuron.py --max_ngen='+NGEN.value+' --offspring_size='+OS.value+\
                ' --start --checkpoint ./checkpoints/checkpoint.pkl\')')
        f.write('\n')
    f.close()
    os.chdir(foldernameOPTstring)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith('init'):
            os.remove(file)
    os.chdir('..')
    shutil.copy('init.py',os.path.join('.',foldernameOPTstring,'init.py'))
    global zfName
    poz=[m.start() for m in re.finditer('_', foldernameOPTstring)]
    zfName = foldernameOPTstring[0:poz[len(poz)-1]]+'_'+datetime.datetime.now().strftime('%Y%m%d%H%M%S')+ '.zip'
    foo = zipfile.ZipFile(zfName, 'w')
    for root, dirs, files in os.walk(os.path.join('.',foldernameOPTstring)):
        if (root==os.path.join('.',foldernameOPTstring,'morphology')) or \
        (root==os.path.join('.',foldernameOPTstring,'config')) or \
        (root==os.path.join('.',foldernameOPTstring,'mechanisms')) or \
        (root==os.path.join('.',foldernameOPTstring,'model')):
            for f in files:
                foo.write(os.path.join(root, f))
        if (root==os.path.join('.',foldernameOPTstring,'checkpoints')) or \
        (root==os.path.join('.',foldernameOPTstring,'figures')):
            foo.write(os.path.join(root))
        if (root==os.path.join('.',foldernameOPTstring)):
            for f in files:
                if f.endswith('.py'):
                    foo.write(os.path.join(root, f))                    
    foo.close()
    os.chdir('../..')
    H2.disabled=True
    BT1.disabled=True
    NGEN.disabled=True
    OS.disabled=True
    BT.disabled=True

    
def createzipPizDaint(_):
    os.chdir('GUIBPOPTONE')
    os.chdir(foldernameOPTstring)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith("ipyparallel.sbatch") or file.startswith("zipfolder.py"):
            os.remove(file)
    with open('zipfolder.py','w') as f:
        f.write('import os\n')
        f.write('import zipfile\n')
        f.write('retval = os.getcwd()\n')
        f.write('print("Current working directory %s" % retval)\n')
        f.write('os.chdir(\'..\')\n')
        f.write('retval = os.getcwd()\n')
        f.write('print("Current working directory %s" % retval)\n')
        f.write('def zipdir(path, ziph):\n')
        f.write('    for root, dirs, files in os.walk(path):\n')
        f.write('        for file in files:\n')
        f.write('            ziph.write(os.path.join(root, file))\n')
        f.write('zipf = zipfile.ZipFile(\'output.zip\', \'w\')\n')
        f.write('zipdir(\''+foldernameOPTstring+'/\', zipf)\n')
    f.close()
    with open('ipyparallel.sbatch','w') as f:
        f.write('#!/bin/bash -l\n')
        f.write('\n')
        f.write('mkdir logs\n')
        f.write('#SBATCH --job-name=bluepyopt_ipyparallel\n')
        f.write('#SBATCH --error=logs/ipyparallel_%j.log\n')
        f.write('#SBATCH --output=logs/ipyparallel_%j.log\n')
        f.write('#SBATCH --partition=normal\n')
        f.write('#SBATCH --constraint=mc\n')
        f.write('\n')
        f.write('export PMI_NO_FORK=1\n')
        f.write('export PMI_NO_PREINITIALIZE=1\n')
        f.write('export PMI_MMAP_SYNC_WAIT_TIME=300 \n')
        f.write('set -e\n')
        f.write('set -x\n')
        f.write('\n')
        f.write('module load daint-mc cray-python/3.8.2.1 PyExtensions/python3-CrayGNU-20.11\n')
        f.write("module load intel/19.0.1.144\n")
        f.write('module use /apps/hbp/ich002/hbp-spack-deployments/softwares/27-10-2020/install/modules/tcl/cray-cnl7-haswell\n')
        f.write('module load neuron/8.0a\n')
        f.write('module load py-bluepyopt\n')
        f.write('nrnivmodl mechanisms\n')
        f.write('\n')
        f.write('export USEIPYP=1\n')
        f.write('export IPYTHONDIR="`pwd`/.ipython"\n')
        f.write('export IPYTHON_PROFILE=ipyparallel.${SLURM_JOBID}\n')
        f.write('ipcontroller --init --sqlitedb --ip=\'*\' --profile=${IPYTHON_PROFILE} &\n')
        f.write('sleep 30\n')
        f.write('srun ipengine --profile=${IPYTHON_PROFILE} &\n')
        f.write('CHECKPOINTS_DIR="checkpoints"\n')
        f.write('python opt_neuron.py --offspring_size='+OS.value+\
                ' --max_ngen='+NGEN.value+' --start --checkpoint "${CHECKPOINTS_DIR}/checkpoint.pkl"\n') 
        f.write('python zipfolder.py')
        f.write('\n')
    f.close()
    os.chdir(foldernameOPTstring)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith('ipyparallel.sbatch') or file.startswith("zipfolder.py"):
            os.remove(file)
    os.chdir('..')
    shutil.copy('ipyparallel.sbatch',os.path.join('.',foldernameOPTstring,'ipyparallel.sbatch'))
    shutil.copy('zipfolder.py',os.path.join('.',foldernameOPTstring,'zipfolder.py'))
    global zfNamePiz
    poz=[m.start() for m in re.finditer('_', foldernameOPTstring)]
    zfNamePiz = foldernameOPTstring[0:poz[len(poz)-1]]+'_'+datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.zip'
    foo = zipfile.ZipFile(zfNamePiz, 'w')
    for root, dirs, files in os.walk(os.path.join('.',foldernameOPTstring)):
        if (root==os.path.join('.',foldernameOPTstring,'morphology')) or \
        (root==os.path.join('.',foldernameOPTstring,'config')) or \
        (root==os.path.join('.',foldernameOPTstring,'mechanisms')) or \
        (root==os.path.join('.',foldernameOPTstring,'model')):
            for f in files:
                foo.write(os.path.join(root, f))
        if (root==os.path.join('.',foldernameOPTstring,'checkpoints')) or \
        (root==os.path.join('.',foldernameOPTstring,'figures')):
            foo.write(os.path.join(root))
        if (root==os.path.join('.',foldernameOPTstring)):
            for f in files:
                if f.endswith('.py') or f.endswith('.sbatch'):
                    foo.write(os.path.join(root, f))                    
    foo.close()
    os.chdir('../..')
    H2.disabled=True
    BT1.disabled=True
    NGEN.disabled=True
    OS.disabled=True
    BTCSCS.disabled=True
    
BT = Button()
BT.description = 'Create zip file'
BT.background_color='gainsboro'
BT.border_color='black'
BT.width='20%'
BT.on_click(createzip)
BTCSCS = Button()
BTCSCS.description = 'Create Piz Daint zip file'
BTCSCS.background_color='gainsboro'
BTCSCS.border_color='black'
BTCSCS.layout.width='30%'
BTCSCS.on_click(createzipPizDaint)

FileNotFoundError: [Errno 2] No such file or directory: 'CA1_pyr_cACpyr_mpg141208_B_idA_20170915151855'

In [ ]:
logging.captureWarnings(False)

In [ ]:
username=''

def checkloginNSG(_):
    username=username_widget.value
    password=password_widget.value
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/'+username
    r = requests.get(URL, auth=(username, password),headers=headers,verify=False)
    root = xml.etree.ElementTree.fromstring(r.text)
    a=0
    for child in root:
        if child.tag == 'displayMessage':
            a=1
            msg=child.text
    if a==0:
        msg='Authenticated successfully'
        msg_widget.disabled=True
        buttonlogin.disabled=True
        username_widget.disabled=True
        password_widget.disabled=True
    msg_widget.value=msg
    msg_widget.layout.display=''

username_widget = Text(description='Username:')
username_widget.layout.width='33%'
password_widget = Password(description='Password:')
buttonlogin = Button()
buttonlogin.description = 'Login NSG'
buttonlogin.background_color='gainsboro'
buttonlogin.border_color='black'
buttonlogin.width='20%'
buttonlogin.on_click(checkloginNSG)
msg_widget = Text()
msg_widget.layout.width='40%'
msg_widget.layout.display='none'
formMNSG = VBox(children=[username_widget,password_widget,buttonlogin,msg_widget])
formMNSG.layout.display=''

NNODES=Text()
NNODES.description='nr nodes'
NNODES.value='2'
NNODES.width=190
NCORES=Text()
NCORES.description='nr cores'
NCORES.value='10'
NCORES.width=190
RT=Text()
RT.description='runtime'
RT.value='1'
RT.width=190
file = open("Work-in-progress3.png", "rb")
image = file.read()
IN = Image(value=image,format='png',width=400)
IN.layout.display='none'
STN = Text()
STN.description='status'
STN.value=''
STN.disabled=True
STN.width=75
STN.layout.display='none'

TT=Text()
TT.description='Title:'
Project=Text()
Project.description='proj'
Project.value=' '
Project.width=190
NNODESCSCS=Text()
NNODESCSCS.description='nr nodes'
NNODESCSCS.value='6'
NNODESCSCS.width=190
NCORESCSCS=Text()
NCORESCSCS.description='nr cores'
NCORESCSCS.value='24'
NCORESCSCS.width=190
RTCSCS=Text()
RTCSCS.description='runtime'
RTCSCS.value='120m'
RTCSCS.width=190
file = open("Work-in-progress3.png", "rb")
image = file.read()
INCSCS = Image(value=image,format='png',width=400)
INCSCS.layout.display='none'
STNCSCS = Text()
STNCSCS.description='status'
STNCSCS.value=''
STNCSCS.disabled=True
STNCSCS.width=75
STNCSCS.layout.display='none'


def runNSG(_):
    RunSNSG.disabled=True
    IN.layout.display=''
    os.chdir('GUIBPOPTONE')
    os.chdir(foldernameOPTstring)
    global CRA_USER, PASSWORD, KEY, URL, TOOL
    username=username_widget.value
    password=password_widget.value
    CRA_USER = username
    PASSWORD = password
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1'
    TOOL = 'BLUEPYOPT_TG'
    
    os.popen('pwd').read()
    os.popen('ls -ltr').read()
    headers = {'cipres-appkey' : KEY}
    payload = {'tool' : TOOL, 'metadata.statusEmail' : 'false', 'vparam.number_cores_' : NCORES.value, \
               'vparam.number_nodes_' : NNODES.value,\
               'vparam.runtime_' : RT.value, \
               'vparam.filename_': 'init.py'}
    files = {'input.infile_' : open(zfName,'rb')}

    r = requests.post('{}/job/{}'.format(URL, CRA_USER), auth=(CRA_USER, PASSWORD), data=payload,\
                      headers=headers, files=files,verify=False)
    
    root = xml.etree.ElementTree.fromstring(r.text)

    global outputuri, selfuri
    for child in root:
        if child.tag == 'resultsUri':
            for urlchild in child:
                if urlchild.tag == 'url':
                    outputuri = urlchild.text
        if child.tag == 'selfUri':
            for urlchild in child:
                if urlchild.tag == 'url':
                    selfuri = urlchild.text
    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers,verify=False)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag=='jobHandle':
            jobname=child.text
    access_token = clb_oauth.get_token()
    headers = {'Authorization': 'Bearer {}'.format(access_token)}
    OIDC_OP_USER_ENDPOINT = "https://iam.ebrains.eu/auth/realms/hbp/protocol/openid-connect/userinfo"
    res = requests.get(url=OIDC_OP_USER_ENDPOINT, headers=headers)
    os.chdir('../..')
    filetoupload=str("listjobsNSG_"+res.json()['preferred_username']+".txt")
    with open(filetoupload, "a") as myfile:
        myfile.write(jobname)
        myfile.write(' ')
        myfile.write(zfName[0:zfName.find('.zip')])
        myfile.write('\n')
    myfile.close()

    myboxUNSG.children[1].layout.display=''

    
def checksimnsg(CheckSNSG):
    RunSNSG.disabled=True
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers,verify=False)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag == 'jobStage':
            jobtodisp=child.text
        if child.tag == 'terminalStage':
            jobstatus = child.text
    STN.layout.display=''
    STN.value=jobtodisp
    if jobstatus=='true':
        storeto_path1 = os.path.join("BluePyOptOne")
        if os.path.isdir(storeto_path1)==False:
            os.mkdir(storeto_path1)
        os.chdir(storeto_path1)
        storeto_path2 = os.path.join("resultsNSG")
        if os.path.isdir(storeto_path2)==False:
            os.mkdir(storeto_path2)
        os.chdir(storeto_path2)
        
        access_token = clb_oauth.get_token()
        headers2 = {'Authorization': 'Bearer {}'.format(access_token)}
        OIDC_OP_USER_ENDPOINT = "https://iam.ebrains.eu/auth/realms/hbp/protocol/openid-connect/userinfo"
        res = requests.get(url=OIDC_OP_USER_ENDPOINT, headers=headers2)
        
        storeto_path3 = str(os.path.join(res.json()['preferred_username']))
        if os.path.isdir(storeto_path3)==False:
            os.mkdir(storeto_path3)
        os.chdir(storeto_path3)
        
        storeto = os.path.join(zfName[0:zfName.find('.zip')])
        os.mkdir(storeto)
        os.chdir(storeto)
        KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
        headers = {'cipres-appkey' : KEY}
        r = requests.get(outputuri,headers=headers, auth=(CRA_USER, PASSWORD),verify=False)
        globaldownloadurilist = []
        root = xml.etree.ElementTree.fromstring(r.text)
        for child in root:
            if child.tag == 'jobfiles':
                for jobchild in child:
                    if jobchild.tag == 'jobfile':
                        for downloadchild in jobchild:
                            if downloadchild.tag == 'downloadUri':
                                for attchild in downloadchild:
                                    if attchild.tag == 'url':
                                        globaldownloadurilist.append(attchild.text)

        # download all output files
        for downloaduri in globaldownloadurilist:
            r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers,verify=False)
            d = r.headers['content-disposition']
            filename_list = re.findall('filename=(.+)', d)
            for filename in filename_list:
                with open(filename, "wb") as fd:
                    for chunk in r.iter_content():
                        fd.write(chunk)  
        os.chdir('../../../..')
        STN.layout.display='none'
        IN.layout.display='none'
        CheckSNSG.layout.display='none'
        RunSNSG.disabled=False
        runningUNSG=False

        
def rununicoreCSCS(_):
    RunSCSCS.disabled=True
    INCSCS.layout.display=''
    os.chdir('GUIBPOPTONE')
    os.chdir(foldernameOPTstring)
    with open(zfNamePiz, 'rb') as content_file:
        content = content_file.read()
    fjob1 = {'To': zfNamePiz, 'Data':content}
    inputs = [fjob1]
    if TT.value=='':
        jobname='UNICORE_Job'
    else:
        jobname=TT.value
    job = {}
    job = {"Executable": "unzip "+zfNamePiz+"; cd "+foldernameOPTstring+"; chmod +rx *.sbatch; ./ipyparallel.sbatch",
           "Name": jobname,
           "Resources": {"Nodes": NNODESCSCS.value,\
                         "CPUsPerNode": NCORESCSCS.value, \
                         "Runtime": RTCSCS.value,\
                         "Project": Project.value,\
                         "NodeConstraints" : "mc",\
                        }, 
          }
    auth = get_oidc_auth() 
    base_url = get_sites()['DAINT-CSCS']['url']
    global job_url
    job_url = submit(base_url + '/jobs', job, auth, inputs)
    os.chdir('../..')
    if job_url!='':
        myboxUCSCS.children[1].layout.display=''
    else:
        INCSCS.layout.display='none'
        myboxUCSCS.children[0].disabled=False

        
def assign_file_types(filename):
    """ assign file types based on file name and extension """
    fname, extension = os.path.splitext(filename)
    if extension == '.txt':
        filetype = 'text/plain'
    else:
        filetype = 'application/unknown'
    return filetype


def checksimCSCS(CheckCSCS):
    RunSCSCS.disabled=True
    auth = get_oidc_auth() 
    job_properties = get_properties(job_url, auth)
    STNCSCS.layout.display=''
    STNCSCS.value=job_properties['status']
    if STNCSCS.value=='SUCCESSFUL':
        auth = get_oidc_auth()
        wd = get_working_directory(job_url, auth)
        output_files = list_files(wd, auth)
        storeto_path = os.path.join('results')
        if os.path.isdir(storeto_path)==False:
            os.mkdir(storeto_path)
        os.chdir(storeto_path)
        storeto = str(os.path.join('output_'+job_properties['submissionTime']))
        os.mkdir(storeto)
        os.chdir(storeto)
        for file_path in output_files:
            _, f = os.path.split(file_path)
            if (f=='stderr') or (f=='stdout') or (f=='output.zip'):
                filetype = assign_file_types(f)
                content = get_file_content(wd+"/files"+file_path,auth)
                with open(f,"wb") as local_file:
                    local_file.write(content)
        os.chdir('../..')
        STNCSCS.layout.display='none'
        INCSCS.layout.display='none'
        CheckSCSCS.layout.display='none'
        myboxUCSCS.children[1].layout.display='none'

        
# =============================================== SERVICE ACCOUNT ======================================================       
def run_on_service_account_pizdaint(_):
    box_SA.children[0].disabled = True
    INCSCS.layout.display=''
    
    SUBMIT_URL = 'https://bspsa.cineca.it/jobs/pizdaint/' 
    command = "unzip "+zfNamePiz+"; cd "+foldernameOPTstring+"; chmod +rx *.sbatch; ./ipyparallel.sbatch"
    payload = {
        'command': command, 
        'node_number': str(NNODESCSCS.value),
        'core_number': str(NCORESCSCS.value),
        'runtime': float(runtimesa.value),
    }
    os.chdir('GUIBPOPTONE')
    os.chdir(foldernameOPTstring)
    job_file = {'file': open(zfNamePiz, 'rb')}
    headers = get_oidc_auth()
    headers.update({'Content-Disposition':'attachment;filename=' + zfNamePiz})
    headers.update({'payload':json.dumps(payload)})
    r = requests.post(url=SUBMIT_URL, headers=headers, files=job_file)
    os.chdir('../..')
    
    if r.status_code == 201:
        job = r.json()
        global jobid
        jobid = job['job_id']
        box_SA.children[1].layout.display = ''
    else:
        INCSCS.layout.display = 'none'
        box_SA.children[0].disabled = False
        

def check_on_service_account_pizdaint(_):
    box_SA.children[1].disabled = True
    CHECK_JOB_URL = 'https://bspsa.cineca.it/jobs/pizdaint/bsp_pizdaint_01/' + jobid + '/'
    FILES_URL = 'https://bspsa.cineca.it/files/pizdaint/bsp_pizdaint_01/'
    
    headers = get_oidc_auth() 
    
    r = requests.get(url=CHECK_JOB_URL, headers=headers)
    if r.status_code == 200:
        job = r.json()
        STNCSCS.layout.display=''
        STNCSCS.value=job['stage']    

        if job['stage'] == 'SUCCESSFUL':
            r = requests.get(url=FILES_URL + jobid + '/', headers=headers)
            if r.status_code == 200:
                files_list = r.json()
                storeto_path = os.path.join('resultsPizdaint-SA')
                if os.path.isdir(storeto_path)==False:
                    os.mkdir(storeto_path)
                os.chdir(storeto_path)
                storeto = str(os.path.join('output_'+job['init_date']))
                os.mkdir(storeto)
                os.chdir(storeto)
                
                for f in files_list:         
                    if f == '/stderr' or f == '/stdout' or f == '/output.zip':
                        filename = f[1:]
                        file_type = assign_file_types(f)
                        DOWNLOAD_FILE_URL = FILES_URL + jobid + f + '/'
                        r = requests.get(url=DOWNLOAD_FILE_URL, headers=headers)
                        if r.status_code == 200:
                            with open(filename, 'wb') as local_file:
                                local_file.write(r.content)
                                
                os.chdir('../..')
                STNCSCS.layout.display = 'none'
                INCSCS.layout.display = 'none'
                checksa_button.layout.display = 'none'
                box_SA.children[1].layout.display = 'none'
        else:
            box_SA.children[1].disabled = False

# =======================================================================================================================

RunSNSG = Button()
RunSNSG.description = 'Run NSG simulation'
RunSNSG.background_color='gainsboro'
RunSNSG.border_color='black'
RunSNSG.width='20%'
RunSNSG.on_click(runNSG)
CheckSNSG = Button(description = 'Check NSG simulation')
CheckSNSG.layout.width='25%'
CheckSNSG.background_color='gainsboro'
CheckSNSG.border_color='black'
CheckSNSG.on_click(checksimnsg)
buttonSNSG =[RunSNSG, CheckSNSG]
myboxUNSG =HBox(children=buttonSNSG)
myboxUNSG.children[0].layout.display=''
myboxUNSG.children[1].layout.display='none'

RunSCSCS = Button()
RunSCSCS.description = 'Run PizDaint simulation'
RunSCSCS.background_color='gainsboro'
RunSCSCS.border_color='black'
RunSCSCS.layout.width='30%'
RunSCSCS.on_click(rununicoreCSCS)
CheckSCSCS = Button()
CheckSCSCS.description = 'Check PizDaint simulation'
CheckSCSCS.background_color='gainsboro'
CheckSCSCS.border_color='black'
CheckSCSCS.layout.width='30%'
CheckSCSCS.on_click(checksimCSCS)
buttonSCSCS =[RunSCSCS, CheckSCSCS]
myboxUCSCS = VBox(children=buttonSCSCS)
myboxUCSCS.children[0].layout.display=''
myboxUCSCS.children[1].layout.display='none'


# ======================================== SERVICE ACCOUNT Accordion Button ============================================

runtimesa = Text(description='Runtime')
runtimesa.value='2.0'
runtimesa.width=190
runsa_button = Button(description='Run Service Account/PizDaint simulation')
runsa_button.background_color = 'gainsboro'
runsa_button.border_color = 'black'
runsa_button.layout.width='30%'
runsa_button.on_click(run_on_service_account_pizdaint)
checksa_button = Button(description='Check Service Account/PizDaint simulation')
checksa_button.background_color = 'gainsboro'
checksa_button.border_color = 'black'
checksa_button.layout.width='30%'
checksa_button.layout.display = 'none'
checksa_button.on_click(check_on_service_account_pizdaint)
buttonsa=[runsa_button, checksa_button]
box_SA = VBox(children=buttonsa)

# ======================================================================================================================


accordion = Accordion(children=[VBox([formMNSG,NNODES,NCORES,RT, STN,IN,BT,myboxUNSG]),
                                VBox([TT,Project,NNODESCSCS,NCORESCSCS,RTCSCS, STNCSCS,INCSCS,BTCSCS,myboxUCSCS],layout=Layout(width='100%')),
                                VBox([NNODESCSCS,NCORESCSCS,runtimesa,STNCSCS,INCSCS,BTCSCS,box_SA],layout=Layout(width='100%'))])
accordion.set_title(0, 'NSG')
accordion.set_title(1, 'PizDaint')
accordion.set_title(2, 'Service Account - Pizdaint')        # SERVICE ACCOUNT
accordion

In [ ]:
from IPython.display import HTML

HTML('''
<button class="toggle_button" onclick="toggleCode(this)">Show Code</button>
<script>
    function toggleCode(button) {
        boxes = button.closest(".jp-Notebook").querySelectorAll(".jp-Cell-inputWrapper");
        if (boxes[0].style.display == "none") {
            boxes.forEach(box => box.style.display = '');
            button.innerHTML = "Hide Code";
        } else {
            boxes.forEach(box => box.style.display = 'none');
            button.innerHTML = "Show Code";
        }
    }
    document.querySelectorAll(".jp-Cell-inputWrapper").forEach(box => box.style.display = 'none');
</script>
''')